In [1]:
!pip install pyngrok diffusers peft


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 35.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 21.5 MB/s eta 0:00:00


In [2]:
from pyngrok import ngrok


ngrok.set_auth_token("2p6aR2UdbsZyxb8oknlZ6RFZljv_4Kkbd8Yr83gZB5N3YdVjF")


In [ ]:
from flask import Flask, request, jsonify
from io import BytesIO
import base64
from PIL import Image
import torch
from diffusers import StableDiffusionPipeline
from pyngrok import ngrok

app = Flask(__name__)


def load_model_with_lora_weights(lora_weights_path, model_name="runwayml/stable-diffusion-v1-5"):
    try:
        
        pipe = StableDiffusionPipeline.from_pretrained(
            model_name,
            torch_dtype=torch.float16
        ).to("cuda" if torch.cuda.is_available() else "cpu")
        print("Base model loaded successfully.")

        
        print("Loading LoRA weights...")
        pipe.load_lora_weights(lora_weights_path)
        print("LoRA weights applied successfully.")

        return pipe
    except Exception as e:
        print(f"Error loading model or LoRA weights: {e}")
        raise


lora_weights_path = "/kaggle/input/lora-weights/pytorch_lora_weights (1000).safetensors"


pipe = load_model_with_lora_weights(lora_weights_path)


def generate(pipeline, prompt, seed, num_inference_steps=35, num_images=1):
    """
    Generate one or more unique images using a Stable Diffusion pipeline.

    Args:
        pipeline: The DiffusionPipeline object.
        prompt (str): The text prompt to generate images.
        seed (int): The initial seed for reproducibility.
        num_inference_steps (int): The number of inference steps. Default is 35.
        num_images (int): The number of unique images to generate. Default is 1.

    Returns:
        List of generated images.
    """
    
    if isinstance(prompt, str):
        prompt = [prompt] * num_images  # Duplicate the prompt for batch generation

    images = []
    for i in range(num_images):
        
        generator = torch.Generator(device="cuda").manual_seed(seed + i)

        
        result = pipeline(prompt[i % len(prompt)], num_inference_steps=num_inference_steps, generator=generator)
        images.append(result.images[0])

    return images


@app.route("/generate", methods=["POST"])
def generate_image():
    try:
        
        data = request.json
        prompt = data.get("prompt")
        seed = data.get("seed", 42)
        num_inference_steps = data.get("num_inference_steps", 35)
        num_images = data.get("num_images", 1)

        if not prompt:
            return jsonify({"error": "Prompt is required"}), 400

        
        images = generate(pipe, prompt, seed, num_inference_steps, num_images)

       
        encoded_images = []
        for image in images:
            buffered = BytesIO()
            image.save(buffered, format="PNG")
            img_str = base64.b64encode(buffered.getvalue()).decode("utf-8")
            encoded_images.append(img_str)

        
        return jsonify({"images": encoded_images})
    except Exception as e:
        return jsonify({"error": str(e)}), 500


if __name__ == "__main__":
    try:
        
        public_url = ngrok.connect(5000)
        print(f"Public URL: {public_url}")

        
        app.run(port=5000)

    except Exception as e:
        print(f"Error starting Flask app: {e}")
    finally:
        
        ngrok.disconnect(public_url)
        print("ngrok tunnel closed.")


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

text_encoder/config.json:   0%|          | 0.00/617 [00:00<?, ?B/s]

...

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Base model loaded successfully.
Loading LoRA weights...
LoRA weights applied successfully.
Public URL: NgrokTunnel: "https://163e-34-31-102-43.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


  0%|          | 0/35 [00:00<?, ?it/s]